
YOU HAVE TO UPLOAD THE REQUIRED VERSION OF 'youtube.apk' TO YOUR DRIVE


File id can be found by..
 1. Open the drive file in chrome
 2. Change accessibility to 'Anyone with link can view'
 3. Copy the id and paste it in next cell

https://drive.google.com/file/d/[file_id]/view?usp=sharing

If this works for you please send feedback by ⭐ project and following.

My GitHub : https://github.com/Jarvis-Ank

In [1]:
id="[file_id]"

Run the below script only after adding the id !!!!!!!!!

In [2]:
!apt update
!apt upgrade

#Drive downloader
!pip install gdown 
!sleep 4

# Creating Directories
print("Creating Directories....")
!mkdir R_V
%cd R_V
!sleep 4

#Zulu JDK-17 download and install
!wget https://cdn.azul.com/zulu/bin/zulu17.34.19-ca-jdk17.0.3-linux_amd64.deb
!sleep 5

!sudo apt install ./zulu17.34.19-ca-jdk17.0.3-linux_amd64.deb
!java --version


!sleep 5

#           ______-----                Downloading  Required files..         -------_______

#       revanced integrations
!wget https://github.com/revanced/revanced-integrations/releases/download/v0.20.1/app-release-unsigned.apk

#       revanced Patches
!wget https://github.com/revanced/revanced-patches/releases/download/v2.8.1/revanced-patches-2.8.1.jar

#       revanced CLI
!wget https://github.com/revanced/revanced-cli/releases/download/v2.3.1/revanced-cli-2.3.1-all.jar

#       Micro-G  
#---> Change this Link if you want..
!wget https://github.com/Jarvis-Ank/Re-Vanced/blob/main/microG_0.2.24.220220.apk

!sleep 6


# id=input("Enter the file id from Google drive : ")

print(id)
!sleep 2
!gdown $id


#           ______-----             Renaming files..         -------_______

print("Renaming Files..")

!mv app-release-unsigned.apk revanced-integrations.apk
!mv revanced-cli-2.3.1-all.jar revanced-cli.jar
!mv revanced-patches-2.8.1.jar revanced-patches.jar
!mv microG_0.2.24.220220.apk microg.apk
# !mv youtube_17.22.36.apk youtube.apk
!sleep 4


#           ______-----             Building revanced.apk        -------_______

!java -jar revanced-cli.jar -a youtube.apk -c -o revanced.apk -b revanced-patches.jar -m revanced-integrations.apk


!cd ..
!pwd

print("SucessFull, check if revanced.apk is in the folder 'R_V'")

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,871 kB]
Get:14 http://archi

If you need to rebuild with different apk please remove created folders by executing cell below. otherwise it'll be hard to work with.

In [ ]:

#  ______-----  Execute to remove previous files..         -------_______

print("Removing Folders")
%rm -rf R_V

In [ ]:
# %cd ..